# Reading Data from csv file and doing basic data manipulation

In [22]:
import pandas as pd
df = pd.read_csv("1.csv", index_col=0)

## Get data for individual Trips

In [23]:
for tripID in set(df['tripID'].values):
    df_trip = df[df['tripID']==tripID]

## Converting Raw ECompass data to engineering units

In [24]:
import numpy as np
# Convert Accelerometer reading to g units, 8bit data with a range of +-2g #
def convert_acc(x):
    x = int(x,16)
    if x>127:
        x = x-256
    return np.float64(x*2/128)

# Convert a single row of Accelerometer data to engineering units and return a df #
def convert_acc_row(row):
    # Initially the data was gathered without magnetometer, so check length for identification #
    data_list = []
    for i in range(0, len(row), 6):
        x = convert_acc(row[i:i+2])
        y = convert_acc(row[i+2:i+4])
        z = convert_acc(row[i+4:i+6])
        
        data_list.append([x,y,z])
    return pd.DataFrame(columns=['ax','ay','az'], data=data_list)

df['accData'] = df['accData'].apply(convert_acc_row)
print(df['accData'].loc[0])

## Storing DataFrame in hdf file

In [28]:
df.to_hdf("data.hdf5", "first")

C:\Miniconda3\lib\site-packages\pandas\core\generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['timeStamp', 'accData']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
